In [4]:
import docx
import re
import xlsxwriter
import cv2
import fnmatch
import numpy as np
import pandas as pd
import pytesseract
from wand.image import Image as wi
import os
import matplotlib.pyplot as plt
import PyPDF2
pytesseract.pytesseract.tesseract_cmd = r"C:\Users\caio.santos\AppData\Local\Tesseract-OCR\tesseract.exe"


In [5]:

def extnum_from_DF(df):
    rex1 = re.compile('R\$\s?\d+?\,?\d?\d?\d?\.?\d?\d?\sm?b?illion')
    rex2 = re.compile('\d+?\.?\d?\d?\%')
    dic = []

    for t in df['Texto']:
        lis = []
        if rex1.search(t):

            for num in rex1.findall(t):
                lis.append(num)
        if rex2.search(t):
            for num in rex2.findall(t):
                lis.append(num)

        dic.append(lis)

    return df.join(pd.DataFrame(dic))


def doc_to_excel(filepath, excelname,padrão=None):
    doc = docx.Document(filepath)
    excel = xlsxwriter.Workbook(excelname + '.xlsx')
    sh = excel.add_worksheet()
    row = 0

    if padrão == None:
        rex1 = re.compile('R\$\s?\d+?\,?\d?\d?\d?\.?\d?\d?\sm?b?illion')
        rex2 = re.compile('\d+?\.?\d?\d?\%')
        for p in doc.paragraphs:
            col = 0

            if rex1.search(p.text):
                row += 1
                sh.write(row-1, col, p.text)


                for num in rex1.findall(p.text):
                    col += 1
                    sh.write(row-1, col, num)

                if rex2.search(p.text):
                    for num in rex2.findall(p.text):
                        col += 1
                        sh.write(row-1, col, num)

    else:
        if padrão == 'R$':
            rex = re.compile('R\$\s?\d+?\,?\d?\d?\d?\.?\d?\d?\sm?b?illion')

        if padrão == '%':
            rex = re.compile('\d+?\.?\d?\d?\%')

        for p in doc.paragraphs:
            col = 0
            if rex.search(p.text):
                row += 1
                sh.write(row-1, col, p.text)

                for num in rex.findall(p.text):
                    col += 1
                    sh.write(row-1, col, num)


    excel.close()

def table_to_excel(filepath, excelname, tableindex):
    doc = docx.Document(filepath)
    excel = xlsxwriter.Workbook(excelname + '.xlsx')
    r = 0

    if tableindex == 'all':
        for t in doc.tables:
            r = 0
            sh = excel.add_worksheet()
            for row in t.rows:
                r += 1
                c = 0
                for cel in row.cells:
                    for p in cel.paragraphs:
                        sh.write(r, c, p.text)
                        c += 1

        excel.close()
        
    elif type(tableindex) == list:
        for t in tableindex:
            tb = doc.tables[t]
            sh = excel.add_worksheet()
            for row in tb.rows:
                r += 1
                c = 0
                for cel in row.cells:
                    for p in cel.paragraphs:
                        sh.write(r, c, p.text)
                        c += 1

            
    else:
        t = doc.tables[tableindex]
        sh = excel.add_worksheet()
        for row in t.rows:
            r += 1
            c = 0
            for cel in row.cells:
                for p in cel.paragraphs:
                    sh.write(r, c, p.text)
                    c += 1

    excel.close()



In [6]:
doc = r"C:\Users\caio.santos\Documents\OFFERING\Orobó\FR\Projeto Orobó - FRE (23.9.2019).docx"

In [7]:
doc_to_excel(doc, r'C:\Users\caio.santos\Documents\eitaa')

In [137]:
file = r"C:\Users\caio.santos\Desktop\LSP\OM\Tera - eRed_Tera_Preliminary OM_English.pdf"

In [138]:
page = []
cont = []

In [139]:
pdf = PyPDF2.PdfFileReader(file,'rb')
nump = pdf.getNumPages()


In [140]:
for p in range(nump):
    text = pdf.getPage(p).extractText()
    page.append(p)
    cont.append(text)

In [141]:
dic = {'page':page, 'conteudo':cont}

In [142]:
df = pd.DataFrame(dic)

In [143]:
page=[]
text=[]

rex = re.compile(r'\n')
for p in range(len(df)):
    string = df['conteudo'][p]
    for line in string.splitlines():
        page.append(p)
        text.append(line)
    




    

In [144]:
dic = dict(page=page,text=text)

In [145]:
df2 = pd.DataFrame(dic)
size=[]
for s in df2['text']:
    size.append(len(s))
df2['size'] = size

In [146]:
df2 = df2[df2['size']>180]

In [147]:
df2

,page,text,size
0,0,The information in this preliminary offering...,307
